In [ ]:
import os

from langgraph.checkpoint.aiosqlite import AsyncSqliteSaver

os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_PROJECT"] = "fridon-ai"

os.environ["PLAYER_TEMPLATE_PATH"] = "./data/templates/"

# from app.core.graph.workflows import graph

In [ ]:
from langchain_openai import ChatOpenAI
from app.core.graph import create_graph
from app.core.plugins.registry import ensure_plugin_registry


plugin_names = ["greeter", "kamino", "jupyter", "symmetry", "coin-price-chart-similarity-search", "coin-technical-analyzer"]
registry = ensure_plugin_registry()

plugins = [registry.plugins[plugin_name]() for plugin_name in plugin_names]

llm = ChatOpenAI(model="gpt-4o", temperature=0)

graph = create_graph(llm, plugins, memory=AsyncSqliteSaver.from_conn_string(":memory:"))


In [ ]:
from IPython.display import display, HTML
import base64


def display_image(image_bytes: bytes, width=1500):
    decoded_img_bytes = base64.b64encode(image_bytes).decode("utf-8")
    html = f'<img src="data:image/png;base64,{decoded_img_bytes}" style="width: {width}px;" />'
    display(HTML(html))

In [ ]:
import nest_asyncio
from langchain_core.runnables.graph import CurveStyle, NodeColors, MermaidDrawMethod

nest_asyncio.apply()  # Required for Jupyter Notebook to run async functions

display_image(
    graph.get_graph().draw_mermaid_png(
        curve_style=CurveStyle.LINEAR,
        node_colors=NodeColors(start="#ffdfba", end="#baffc9", other="#fad7de"),
        wrap_label_n_words=9,
        output_file_path=None,
        draw_method=MermaidDrawMethod.PYPPETEER,
        background_color="white",
        padding=10,
    )
)

In [ ]:
config = {
    "configurable": {
        "thread_id": "bla",
        "wallet_id": "adasdasdad",
        "chat_id": "bla"
    }
}


In [ ]:
async for s in graph.astream(
        {
            "messages": [
                "Hi, how are you?"
            ],
        },
        config,
        stream_mode="values"
):
    if "__end__" not in s:
        s["messages"][-1].pretty_print()

In [ ]:
(await graph.aget_state(config)).values.get("used_agents", [])